### 네이버 뉴스 섹션별 TOPIC 크롤링
**정치, 경제, 사회, 생활/문화, IT/과학, 세계**

In [1]:
from urllib.request import urlopen
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [2]:
# 02_0에서 저장한 섹션별 link 파일 불러오기 
# 
df_menu = pd.read_csv('./crawl_data/naver_news_section.csv',index_col = 0) 
# index_col csv 파일 열어보면 0 1 2 이것들을 index로 처리하겠다

df_menu.head()

,section,link
0,언론사별,https://news.naver.com/?viewType=pc
1,정치,https://news.naver.com/main/main.naver?mode=LS...
2,경제,https://news.naver.com/main/main.naver?mode=LS...
3,사회,https://news.naver.com/main/main.naver?mode=LS...
4,생활/문화,https://news.naver.com/main/main.naver?mode=LS...


#### 경제 섹션을 예시 코드로 작성

In [3]:
df_menu['link'][2]

'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101'

In [4]:
# 경제 섹션 소스 요청
url = df_menu['link'][2]
# res = requests.get(url) # 요청 후 응답객체 변환 
# ConnectionResetError     
# ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))
# 원격 봇이라고 생각하고 연결 종료 
# res

# urlopen() 함수는 진행됨
# res = urlopen(url)
# res.read()

- 위 코드처럼 자동화 봇으로 보고 연결 끊는 경우 해결방법 
    - 서버에 요청시 header를 구성해 추가해 줌 즉, bot이 아님을 증명 
    - header 확인
        - 브라우저에서 해당 사이트 접속 시 생성하는 헤더를 
        - 개발자도구로 확인(브라우저 정보 : User-Agent)

In [5]:
# 소스 가져오기
url = df_menu['link'][2]
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
res = requests.get(url,headers= headers)

In [6]:
html = res.text

In [7]:
# url = df_menu['link'][2]
# html = urlopen(url) 
# 두 번재 위 셀로 해도되고 이 셀로 해도되고 

In [8]:
bs_obj = BeautifulSoup(html,'html.parser')

In [9]:
bs_obj


<!DOCTYPE HTML>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<meta content="600" http-equiv="refresh">
<meta content="width=1106" name="viewport">
<meta content="경제 : 네이버 뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://news.naver.com/main/main.naver?mode=LSD&amp;mid=shm&amp;sid1=101" property="og:url"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" property="og:image">
<meta content="증권, 금융, 부동산, 기업, 국제 등 경제 분야 뉴스 제공" property="og:description"/>
<meta content="네이버" property="og:article:author">
<meta content="summary" name="twitter:card"/>
<meta content="경제 : 네이버 뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" name="tw

- 네이버 뉴스 섹션을 확인하면 각 섹션별 topic이 있고 topic중 헤드라인이 노출
- 섹션에서 수집하는 것 보다 토픽페이지로 이동해서 토픽기사 전체를 크롤링 하면 더 많은 기사를 추출 할 수 있음
- 섹션 페이지에서는 토픽의 title과 link를 크롤링
- 단, 정치 섹션은 구성이 약간 다르므로, 수정해서 사용

In [10]:
# 토픽 추출
#main_content > div > div._persist > div:nth-child(1) > div:nth-child(1) > div.cluster_head > div > div > h2
# h2태그의 class:cluster_head_topic
topic_list = bs_obj.findAll('h2',{'class':'cluster_head_topic'})
len(topic_list) # list로 반환

13

In [11]:
topic_list

[<h2 class="cluster_head_topic">
 <a class="nclicks(cls_eco.clstitle)" href="/main/clusterArticles.naver?id=c_202207262210_00000002&amp;mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=009&amp;aid=0004996700">
 <span class="cluster_head_sub_topic">IMF </span><span class="cluster_head_sub_topic">올해 세계경제성장전망 3.2%로 하향 </span>
 </a>
 </h2>,
 <h2 class="cluster_head_topic">
 <a class="nclicks(cls_eco.clstitle)" href="/main/clusterArticles.naver?id=c_202207221010_00000017&amp;mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=009&amp;aid=0004996293">
 <span class="cluster_head_sub_topic">스타벅스 캐리백 밖으로 던졌다 </span><span class="cluster_head_sub_topic">발암물질 논란 커지자 </span>
 </a>
 </h2>,
 <h2 class="cluster_head_topic">
 <a class="nclicks(cls_eco.clstitle)" href="/main/clusterArticles.naver?id=c_202207261110_00000015&amp;mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=052&amp;aid=0001768809">
 <span class="cluster_head_sub_topic">상가조합원 재건축부담금 폭탄 피한다 </span><span class="cluster_head_sub_topic">"5년 소유·3년 거주" 소규모 정비사업 </sp

In [12]:
topic_list[0].find('a')
topic_list[0].find('a')['href'] # url 추출
topic_list[0].find('a').text.replace('\n','')

'IMF 올해 세계경제성장전망 3.2%로 하향 '

In [13]:
# 전체 토픽 title과 link 출력 
for topic in topic_list:
    a_t = topic.find('a')
    print(a_t.text.replace('\n','')) # 마지막에 a태그가 없는 거는 에러가 남
    print(a_t['href'])
# 마지막 부분에서 a태그가 없는 요소(빈 item)가 있어서 에러가 발생 
# 예외처리 진행


IMF 올해 세계경제성장전망 3.2%로 하향 
/main/clusterArticles.naver?id=c_202207262210_00000002&mode=LSD&mid=shm&sid1=101&oid=009&aid=0004996700
스타벅스 캐리백 밖으로 던졌다 발암물질 논란 커지자 
/main/clusterArticles.naver?id=c_202207221010_00000017&mode=LSD&mid=shm&sid1=101&oid=009&aid=0004996293
상가조합원 재건축부담금 폭탄 피한다 "5년 소유·3년 거주" 소규모 정비사업 
/main/clusterArticles.naver?id=c_202207261110_00000015&mode=LSD&mid=shm&sid1=101&oid=052&aid=0001768809
"2분기 경제성장률" 0.7% "수출 줄었지만 소비 덕봤네" 
/main/clusterArticles.naver?id=c_202207260810_00000003&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000829897
우아한청년들·바로고 소화물배송대행서비스 1호 인증 국토부 
/main/clusterArticles.naver?id=c_202207270610_00000006&mode=LSD&mid=shm&sid1=101&oid=003&aid=0011328359
GM에 대규모 양극재 공급 LG화학 
/main/clusterArticles.naver?id=c_202207270830_00000004&mode=LSD&mid=shm&sid1=101&oid=011&aid=0004080666
JB금융 상반기 순익 3200억 안정적인 2분기 실적 
/main/clusterArticles.naver?id=c_202207261340_00000023&mode=LSD&mid=shm&sid1=101&oid=016&aid=0002020697
용산정비창 아시아 실리콘밸리로 재탄생 "용적률 1500% 이상" 
/main/clusterAr

AttributeError: 'NoneType' object has no attribute 'text'

In [14]:
# 전체 토픽 title과 link 출력 
for topic in topic_list:
    try:
        a_t = topic.find('a')
        print(a_t.text.replace('\n','')) # 마지막에 a태그가 없는 거는 에러가 남
        print(a_t['href'])
    except:
        print('에러발생')
# 마지막 부분에서 a태그가 없는 요소(빈 item)가 있어서 에러가 발생 
# 예외처리 진행


IMF 올해 세계경제성장전망 3.2%로 하향 
/main/clusterArticles.naver?id=c_202207262210_00000002&mode=LSD&mid=shm&sid1=101&oid=009&aid=0004996700
스타벅스 캐리백 밖으로 던졌다 발암물질 논란 커지자 
/main/clusterArticles.naver?id=c_202207221010_00000017&mode=LSD&mid=shm&sid1=101&oid=009&aid=0004996293
상가조합원 재건축부담금 폭탄 피한다 "5년 소유·3년 거주" 소규모 정비사업 
/main/clusterArticles.naver?id=c_202207261110_00000015&mode=LSD&mid=shm&sid1=101&oid=052&aid=0001768809
"2분기 경제성장률" 0.7% "수출 줄었지만 소비 덕봤네" 
/main/clusterArticles.naver?id=c_202207260810_00000003&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000829897
우아한청년들·바로고 소화물배송대행서비스 1호 인증 국토부 
/main/clusterArticles.naver?id=c_202207270610_00000006&mode=LSD&mid=shm&sid1=101&oid=003&aid=0011328359
GM에 대규모 양극재 공급 LG화학 
/main/clusterArticles.naver?id=c_202207270830_00000004&mode=LSD&mid=shm&sid1=101&oid=011&aid=0004080666
JB금융 상반기 순익 3200억 안정적인 2분기 실적 
/main/clusterArticles.naver?id=c_202207261340_00000023&mode=LSD&mid=shm&sid1=101&oid=016&aid=0002020697
용산정비창 아시아 실리콘밸리로 재탄생 "용적률 1500% 이상" 
/main/clusterAr

- base_url : https://news.naver.com
- sub_url : /main/clusterArticles.naver?id=c_202207251130_00000383&mode=LSD&mid=shm&sid1=101&oid=648&aid=0000009369
- 수집된 url은 기본 url이 빠져 있으므로 기본 url을 첨부하여 수집 해야 함

In [15]:
# link에 기본 url 추가
base_url = 'https://news.naver.com'
# 전체 토픽 title과 link 출력 
for topic in topic_list:
    try:
        a_t = topic.find('a')
        print(a_t.text.replace('\n','')) # 마지막에 a태그가 없는 거는 에러가 남
        print(base_url + a_t['href'])
    except:
        print('에러발생')
# 마지막 부분에서 a태그가 없는 요소(빈 item)가 있어서 에러가 발생 
# 예외처리 진행

IMF 올해 세계경제성장전망 3.2%로 하향 
https://news.naver.com/main/clusterArticles.naver?id=c_202207262210_00000002&mode=LSD&mid=shm&sid1=101&oid=009&aid=0004996700
스타벅스 캐리백 밖으로 던졌다 발암물질 논란 커지자 
https://news.naver.com/main/clusterArticles.naver?id=c_202207221010_00000017&mode=LSD&mid=shm&sid1=101&oid=009&aid=0004996293
상가조합원 재건축부담금 폭탄 피한다 "5년 소유·3년 거주" 소규모 정비사업 
https://news.naver.com/main/clusterArticles.naver?id=c_202207261110_00000015&mode=LSD&mid=shm&sid1=101&oid=052&aid=0001768809
"2분기 경제성장률" 0.7% "수출 줄었지만 소비 덕봤네" 
https://news.naver.com/main/clusterArticles.naver?id=c_202207260810_00000003&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000829897
우아한청년들·바로고 소화물배송대행서비스 1호 인증 국토부 
https://news.naver.com/main/clusterArticles.naver?id=c_202207270610_00000006&mode=LSD&mid=shm&sid1=101&oid=003&aid=0011328359
GM에 대규모 양극재 공급 LG화학 
https://news.naver.com/main/clusterArticles.naver?id=c_202207270830_00000004&mode=LSD&mid=shm&sid1=101&oid=011&aid=0004080666
JB금융 상반기 순익 3200억 안정적인 2분기 실적 
https://news.naver.com/m

### 수집한 data를 DataFrame으로 생성

In [16]:
# 수집 내용 list에 저장
topic_title = []
topic_link = []
# link에 기본 url 추가
base_url = 'https://news.naver.com'
# 전체 토픽 title과 link 출력 
for topic in topic_list:
    try:
        a_t = topic.find('a')
        topic_title.append(a_t.text.replace('\n','')) # 마지막에 a태그가 없는 거는 에러가 남
        topic_link.append(base_url + a_t['href'])
    except:
        print('에러발생')
# 마지막 부분에서 a태그가 없는 요소(빈 item)가 있어서 에러가 발생 
# 예외처리 진행

에러발생


In [17]:
topic_title


['IMF 올해 세계경제성장전망 3.2%로 하향 ',
 '스타벅스 캐리백 밖으로 던졌다 발암물질 논란 커지자 ',
 '상가조합원 재건축부담금 폭탄 피한다 "5년 소유·3년 거주" 소규모 정비사업 ',
 '"2분기 경제성장률" 0.7% "수출 줄었지만 소비 덕봤네" ',
 '우아한청년들·바로고 소화물배송대행서비스 1호 인증 국토부 ',
 'GM에 대규모 양극재 공급 LG화학 ',
 'JB금융 상반기 순익 3200억 안정적인 2분기 실적 ',
 '용산정비창 아시아 실리콘밸리로 재탄생 "용적률 1500% 이상" ',
 '현대제철 2분기 영업익 8221억원 전년比 51% 증가 ',
 '월마트 실적 전망 하향 인플레로 소비 위축되나 ',
 '"우리은행 횡령" 700억 육박 금감원 "내부통제 미흡" ',
 'LG엔솔 中에 "배터리 리사이클" 합작사 설립 ']

In [18]:
topic_link

['https://news.naver.com/main/clusterArticles.naver?id=c_202207262210_00000002&mode=LSD&mid=shm&sid1=101&oid=009&aid=0004996700',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207221010_00000017&mode=LSD&mid=shm&sid1=101&oid=009&aid=0004996293',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207261110_00000015&mode=LSD&mid=shm&sid1=101&oid=052&aid=0001768809',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207260810_00000003&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000829897',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207270610_00000006&mode=LSD&mid=shm&sid1=101&oid=003&aid=0011328359',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207270830_00000004&mode=LSD&mid=shm&sid1=101&oid=011&aid=0004080666',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207261340_00000023&mode=LSD&mid=shm&sid1=101&oid=016&aid=0002020697',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207261020_00000004&mode=LSD&mi

In [19]:
topic_df = pd.DataFrame({'contents':topic_title, 'URL':topic_link})
topic_df.head()

,contents,URL
0,IMF 올해 세계경제성장전망 3.2%로 하향,https://news.naver.com/main/clusterArticles.na...
1,스타벅스 캐리백 밖으로 던졌다 발암물질 논란 커지자,https://news.naver.com/main/clusterArticles.na...
2,"상가조합원 재건축부담금 폭탄 피한다 ""5년 소유·3년 거주"" 소규모 정비사업",https://news.naver.com/main/clusterArticles.na...
3,"""2분기 경제성장률"" 0.7% ""수출 줄었지만 소비 덕봤네""",https://news.naver.com/main/clusterArticles.na...
4,우아한청년들·바로고 소화물배송대행서비스 1호 인증 국토부,https://news.naver.com/main/clusterArticles.na...


### 위 내용을 결합해서 함수로 구성
- 다른 섹션에도 적용하기 위해
- 함수명 : get_topic(url)
- 반환  : 섹션별 수집 데이터를 dict로 구성 후 반환 

In [20]:
# 함수 구성
def get_topic(url,section):
    
    # 수집 내용 list에 저장
    topic_title = []
    topic_link = []
    
    # 소스 요청해서 가져오기
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
    res = requests.get(url,headers= headers)
    
    # 파서기 생성(bs4 객체 생성)
    html = res.text
    bs_obj = BeautifulSoup(html,'html.parser')
    
    # 토픽 추출
    # h2태그의 class:cluster_head_topic
    topic_list = bs_obj.findAll('h2',{'class':'cluster_head_topic'})
    
    # 최종 수집 항목을 list에 저장 
    # link에 기본 url 추가
    base_url = 'https://news.naver.com'
    # 전체 토픽 title과 link 출력 
    for topic in topic_list:
        try:
            a_t = topic.find('a')
            topic_title.append(a_t.text.replace('\n','')) # 마지막에 a태그가 없는 거는 에러가 남
            topic_link.append(base_url + a_t['href'])
        except:
            print('에러발생')
    # 마지막 부분에서 a태그가 없는 요소(빈 item)가 있어서 에러가 발생 
    # 예외처리 진행
    
    return({'topic':topic_title,'URL':topic_link,'section':section})

In [21]:
# 함수 테스트
get_topic(url,'경제')


에러발생


{'topic': ['금융위 금융안정계정 도입 부실 사전에 차단한다 ',
  '대우조선 경영진 파업 사태 대국민 사과 ',
  '"2분기 경제성장률" 0.7% "수출 줄었지만 소비 덕봤네" ',
  '현대위아 2분기 영업익 531억원 전년比 17.6% ↑ ',
  'LG엔솔 中에 "배터리 리사이클" 합작사 설립 ',
  '식당가 매출 62.1% 확대 장마·폭염에 백화점 발길↑ ',
  '고령층 10명 중 7명 "73세까지 일하고 싶다" ',
  '"우리은행 횡령" 700억 육박 금감원 "내부통제 미흡" ',
  '최태원 SK 회장 "미국에 29조원 추가 투자" ',
  '프레시지 새벽배송 잠정 중단 GS리테일은 빠지고 네이버는 들어오고 ',
  '올해 수입보험료 증가율 1.2% 보증보험 시장 블루오션으로 떠오르나 ',
  'IMF 올해 세계경제성장전망 3.2%로 하향 '],
 'URL': ['https://news.naver.com/main/clusterArticles.naver?id=c_202207261100_00000672&mode=LSD&mid=shm&sid1=101&oid=003&aid=0011326196',
  'https://news.naver.com/main/clusterArticles.naver?id=c_202207261450_00000005&mode=LSD&mid=shm&sid1=101&oid=079&aid=0003668490',
  'https://news.naver.com/main/clusterArticles.naver?id=c_202207260810_00000003&mode=LSD&mid=shm&sid1=101&oid=008&aid=0004775164',
  'https://news.naver.com/main/clusterArticles.naver?id=c_202207261430_00000003&mode=LSD&mid=shm&sid1=101&oid=018&aid=0005277451',
  'https://news.naver.com/main

In [22]:
# 02_0에서 저장한 섹션별 url 파일 가져오기 
df_menu = pd.read_csv('./crawl_data/naver_news_section.csv',index_col = 0)
df_menu.head(8)

,section,link
0,언론사별,https://news.naver.com/?viewType=pc
1,정치,https://news.naver.com/main/main.naver?mode=LS...
2,경제,https://news.naver.com/main/main.naver?mode=LS...
3,사회,https://news.naver.com/main/main.naver?mode=LS...
4,생활/문화,https://news.naver.com/main/main.naver?mode=LS...
5,IT/과학,https://news.naver.com/main/main.naver?mode=LS...
6,세계,https://news.naver.com/main/main.naver?mode=LS...
7,랭킹,https://news.naver.com/main/ranking/popularDay...


In [23]:
# 함수 test
pd.DataFrame(get_topic(df_menu['link'][3],df_menu['section'][3]))

에러발생


,topic,URL,section
0,"""싸이 흠뻑쇼"" 후 확진 방역당국 조사 나섰다",https://news.naver.com/main/clusterArticles.na...,사회
1,"""교사 컴퓨터에 악성코드"" 심어 고교생 2명 입건",https://news.naver.com/main/clusterArticles.na...,사회
2,류삼영 총경 경찰국 신설 졸속통과…권한쟁의심판청구 촉구,https://news.naver.com/main/clusterArticles.na...,사회
3,임시선별검사소 12곳 운영 중 인천시,https://news.naver.com/main/clusterArticles.na...,사회
4,"""14만 전체경찰회의"" 철회 ""국회가 경찰국 시정해줄 것""",https://news.naver.com/main/clusterArticles.na...,사회
5,"국정원에 尹대통령·국회의장 살해 협박글 ""윤석열·김진표 살해하겠다""",https://news.naver.com/main/clusterArticles.na...,사회
6,"""휴대용 선풍기"" 전자파 발암 기준치의 최대 322배",https://news.naver.com/main/clusterArticles.na...,사회
7,14만 전체 경찰회의로 확대 경찰국 반발 격화,https://news.naver.com/main/clusterArticles.na...,사회
8,"""100억 투자금 손실 혐의"" 코인 채굴업체 압수수색 경찰",https://news.naver.com/main/clusterArticles.na...,사회
9,"""경찰국 반대"" 청원 10만명 돌파 ""국회 제출 검토""",https://news.naver.com/main/clusterArticles.na...,사회


## 추출하려하는 모든 섹션에 적용 

In [24]:
# 모든 섹션의 topic을 하나의 dataframe에 저장하기 위해 빈 dataframe을 생성
dict_sub = {}
dict_sub['topic'] = []
dict_sub['url'] = []
dict_sub['section'] = []

topic_df = pd.DataFrame(dict_sub)
topic_df

,topic,url,section


In [25]:
# 섹션별 헤드라인 추출 
for i in range(1,7):
    temp = pd.DataFrame(get_topic(df_menu['link'][i],df_menu['section'][i]))
    topic_df = pd.concat([topic_df,temp],axis=0,ignore_index=True)

에러발생
에러발생
에러발생
에러발생
에러발생
에러발생
에러발생
에러발생


In [26]:
topic_df.head()
topic_df.tail()

,topic,url,section,URL
50,"시진핑과 28일 통화할 듯 ""대만·우크라 문제 논의""",NaN,세계,https://news.naver.com/main/clusterArticles.na...
51,"바이든 한국전 정전일 포고문 ""한미동맹 어느 때보다 강력""",NaN,세계,https://news.naver.com/main/clusterArticles.na...
52,곡물수출 합의후 벌써 2번째 우크라 항만도시 공습,NaN,세계,https://news.naver.com/main/clusterArticles.na...
53,英 총리 후보 TV토론 생방송 중 진행자 실신,NaN,세계,https://news.naver.com/main/clusterArticles.na...
54,아르헨 쓰레기장서 보물찾기 소동 현금 1억 발견,NaN,세계,https://news.naver.com/main/clusterArticles.na...


### 위 결과에는 정치 섹션이 들어오지 않음 - 태그 구성이 다르기 때문
- 정치 섹션도 포함되게 함수 변경
- https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100 를 확인하면 주제 문구가 없음 


In [27]:
# 함수 구성

def get_topic(url,section):
    
    # 수집 내용 list에 저장
    topic_title = []
    topic_link = []
    
    # 소스 요청해서 가져오기
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
    res = requests.get(url,headers= headers)
    
    # 파서기 생성(bs4 객체 생성)
    html = res.text
    bs_obj = BeautifulSoup(html,'html.parser')
    
    
    # section이 정치인 경우는 다른 태그를 추출
    if section == '정치':
        topic_list = bs_obj.findAll('div',{'class':'cluster_foot_inner'})
    else:
        # 토픽 추출
        # h2태그의 class:cluster_head_topic
        topic_list = bs_obj.findAll('h2',{'class':'cluster_head_topic'})
    
   
    
    # 최종 수집 항목을 list에 저장 
    # link에 기본 url 추가
    base_url = 'https://news.naver.com'
    # 전체 토픽 title과 link 출력 
    for topic in topic_list:
        try:
            a_t = topic.find('a')
            topic_title.append(a_t.text.replace('\n','')) # 마지막에 a태그가 없는 거는 에러가 남
            topic_link.append(base_url + a_t['href'])
        except:
            print('에러발생')
    # 마지막 부분에서 a태그가 없는 요소(빈 item)가 있어서 에러가 발생 
    # 예외처리 진행
    
    return({'topic':topic_title,'url':topic_link,'section':section})

In [28]:
# 모든 섹션의 topic을 하나의 dataframe에 저장하기 위해 빈 dataframe을 생성
dict_sub = {}
dict_sub['topic'] = []
dict_sub['url'] = []
dict_sub['section'] = []

topic_df = pd.DataFrame(dict_sub)
topic_df

,topic,url,section


In [29]:
# 섹션별 헤드라인 추출 
for i in range(1,7):
    temp = pd.DataFrame(get_topic(df_menu['link'][i],df_menu['section'][i]))
    topic_df = pd.concat([topic_df,temp],axis=0,ignore_index=True)

에러발생
에러발생
에러발생
에러발생
에러발생
에러발생
에러발생


In [30]:
topic_df.head()
topic_df.tail()

,topic,url,section
65,2024년 이후 국제우주정거장 탈퇴 NASA 꿀벌 로봇,https://news.naver.com/main/clusterArticles.na...,세계
66,객관적·공정한 입장 견지해야 한국 칩4 참여 견제,https://news.naver.com/main/clusterArticles.na...,세계
67,펠로시 美하원의장 대만 방문계획에 전례 없는 강력 경고,https://news.naver.com/main/clusterArticles.na...,세계
68,美 에너지부 LG·GM 합작 배터리 벤처에 3조 금융 지원,https://news.naver.com/main/clusterArticles.na...,세계
69,겸허히 용서 구한다 캐나다 원주민 아동학살 사죄,https://news.naver.com/main/clusterArticles.na...,세계


In [31]:
# 파일로 저장
topic_df.to_csv('./crawl_data/naver_news_topic.csv')